In [39]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from tqdm import tqdm
import numpy as np
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [4]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [2]:
train = pd.read_excel(r'TrainTest.xlsx',sheet_name='Train')
train.head()

,sentences,class
0,how about 3:30,0
1,please let me know when this has been done and...,1
2,"email me the names of the candidates, whom you...",1
3,please pass this info to all of our traders th...,1
4,hey:any interest in king biscuit after work??,0


In [10]:
label_encode = preprocessing.LabelEncoder()
y = label_encode.fit_transform(train['class'])

In [11]:
xtrain, xvalid, ytrain, yvalid = train_test_split(train.sentences.values, y, 
                                                  stratify=y, 
                                                  random_state=42, 
                                                  test_size=0.1, shuffle=True)

In [13]:
print(xtrain.shape),print (xvalid.shape)

(2115,)
(235,)


(None, None)

In [15]:
tf_idf = TfidfVectorizer(min_df=3,  max_features=None, strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), use_idf=1,smooth_idf=1,sublinear_tf=1,stop_words = 'english')

In [17]:
# Fitting TF-IDF 
tf_idf.fit(list(xtrain) + list(xvalid))
xtrain_tf_idf =  tf_idf.transform(xtrain) 
xvalid_tf_idf = tf_idf.transform(xvalid)

In [34]:
logistic_ml = LogisticRegression(C=1.0)
logistic_ml.fit(xtrain_tf_idf, ytrain)
predictions = logistic_ml.predict_proba(xvalid_tf_idf)

In [36]:
valid_predicted = []
for i in range(len(predictions)):
    valid_predicted.append(np.argmax(predictions[i]))

In [40]:
# confusion matrix on Valid Data
sklearn.metrics.confusion_matrix(yvalid, valid_predicted)

array([[112,   8],
       [  9, 106]])

In [41]:
#Checking model on Test Data
test = pd.read_excel(r'TrainTest.xlsx',sheet_name='Test')
test.head()

,sentences,class
0,"andrea,please resend the first three resumes.p...",0
1,cancel my rsvp for carrin patman's lunch tomor...,1
2,"bob zacouras, report to the designated seats i...",1
3,i would like to have a copy of the appraisal. ...,0
4,"american express, please hit the reply button ...",1


In [43]:
xtest = test['sentences']
xtest_tf_idf =  tf_idf.transform(xtest) 

In [44]:
predictions_test = logistic_ml.predict_proba(xtest_tf_idf)

In [47]:
test_predicted = []
for i in range(len(predictions_test)):
    test_predicted.append(np.argmax(predictions_test[i]))

In [48]:
# confusion matrix on Test Data
sklearn.metrics.confusion_matrix(test['class'], test_predicted)

array([[92,  4],
       [22, 78]])